In [1]:
import json
import pandas as pd
import numpy as np
import math

from datetime import datetime, timedelta
from kafka import KafkaConsumer, KafkaProducer

In [2]:

# Kafka consumer configuration for topic simple-moving-average
sma_consumer = KafkaConsumer("simple-moving-average", bootstrap_servers=["localhost:9093"], value_deserializer=lambda x: json.loads(x.decode("utf-8")))


In [3]:

# Kafka producer configuration for topic sma
producer = KafkaProducer(bootstrap_servers=["localhost:9094"], value_serializer=lambda x: json.dumps(x).encode("utf-8"))


In [4]:
AAPL = pd.DataFrame(columns = ['time','cost'] )
GOOG = pd.DataFrame(columns = ['time','cost'] )
TSLA = pd.DataFrame(columns = ['time','cost'] )
AMZN = pd.DataFrame(columns = ['time','cost'] )
FB = pd.DataFrame(columns = ['time','cost'] )

In [5]:
AAPL_sma_pd = pd.DataFrame(columns = ['time','cost'] )
GOOG_sma_pd = pd.DataFrame(columns = ['time','cost'] )
TSLA_sma_pd = pd.DataFrame(columns = ['time','cost'] )
AMZN_sma_pd = pd.DataFrame(columns = ['time','cost'] )
FB_sma_pd = pd.DataFrame(columns = ['time','cost'] )

In [6]:
def print_sma(AAPL_sma, GOOG_sma, TSLA_sma, AMZN_sma, FB_sma):
    print("\n\n\nAt ", datetime.now())
    print("\n-----------------------------------------------------\nSimple Moving Average of instruments are as follows: \n---------------------------------------------------")
    
    #print("AAPL ", "GOOG ", "TSLA ", "AMZN ", "FB ")
    #print(round(AAPL_sma, 2), " " ,round(GOOG_sma, 2), " " ,round(TSLA_sma,2) , " " ,round(AMZN_sma, 2), " " , round(FB_sma, 2) )
    #print("-----------------------------------------------------\n")
    df = pd.DataFrame({
        'AAPL': [round(AAPL_sma, 2)],
        'GOOG': [round(GOOG_sma, 2)],
        'TSLA': [round(TSLA_sma,2)],
        'AMZN': [round(AMZN_sma,2)],
        'FB': [round(FB_sma,2)]
        })
    display(df)
    print("-----------------------------------------------------\n")
    


In [7]:
min_5_window = datetime.now()
min_10_window = datetime.now()

In [8]:
length = {"AAPL": 1, "GOOG": 1, "TSLA":1, "AMZN": 1, "FB":1}

In [9]:
# Send completed sma price after for all the intruments
def sma_price(apple, google, tesla, amazon, facebook):
    sma_price_row = {
        "AAPL": round(apple,2),
        "GOOG": round(google,2),
        "TSLA": round(tesla,2),
        "AMZN": round(amazon,2),
        "FB": round(facebook,2),
        "time": datetime.now().isoformat()
    }
    producer.send("profit-calculate", value=sma_price_row)
    print(f"Message sent to topic -> profit-calculate :\n---------------------------------------------\n{sma_price_row}")
    print("---------------------------------------------\n")


In [ ]:
for count, msg in enumerate(sma_consumer):

    traded_price = msg.value
    #print("Completed Trade Received : ", traded_price["instrument"] ," - ", traded_price["cost"]," - ", traded_price["time"])
    if traded_price["instrument"] == "AAPL" :
      AAPL.loc[len(AAPL.index)] = [ traded_price["time"] , traded_price["cost"] ]
      AAPL_sma_pd.loc[len(AAPL.index)] = [ traded_price["time"] , traded_price["cost"] ]
      #display(AAPL_sma_pd)
    elif traded_price["instrument"] == "GOOG" :
      GOOG.loc[len(GOOG.index)] = [ traded_price["time"] , traded_price["cost"] ]
      GOOG_sma_pd.loc[len(GOOG.index)] = [ traded_price["time"] , traded_price["cost"] ]
      #display(GOOG_sma_pd)
    elif traded_price["instrument"] == "TSLA" :
      TSLA.loc[len(TSLA.index)] = [ traded_price["time"] , traded_price["cost"] ]
      TSLA_sma_pd.loc[len(TSLA.index)] = [ traded_price["time"] , traded_price["cost"] ]
      #display(TSLA_sma_pd)
    elif traded_price["instrument"] == "AMZN" :
      AMZN.loc[len(AMZN.index)] = [ traded_price["time"] , traded_price["cost"] ]
      AMZN_sma_pd.loc[len(AMZN.index)] = [ traded_price["time"] , traded_price["cost"] ]
      #display(AMZN_sma_pd)
    elif traded_price["instrument"] == "FB" :
      FB.loc[len(FB.index)] = [ traded_price["time"] , traded_price["cost"] ]
      FB_sma_pd.loc[len(FB.index)] = [ traded_price["time"] , traded_price["cost"] ]
      #display(FB_sma_pd)
    else:
      print("Instrument not recognised")
         
    #print( pd.to_datetime( AAPL_sma_pd['time'] , format = '%Y-%m-%dT%H:%M:%S.%f') >= datetime.now() - timedelta(minutes=10) )
    #print(type( pd.to_datetime( AAPL_sma_pd['time'] , format = '%Y-%m-%dT%H:%M:%S.%f') ))
    #print(datetime.now() - timedelta(minutes=10))

    AAPL_sma_pd = AAPL_sma_pd[ pd.to_datetime( AAPL_sma_pd['time'] , format = '%Y-%m-%dT%H:%M:%S.%f') >= datetime.now() - timedelta(minutes=10) ]
    GOOG_sma_pd = GOOG_sma_pd[ pd.to_datetime( GOOG_sma_pd['time'] , format = '%Y-%m-%dT%H:%M:%S.%f') >= datetime.now() - timedelta(minutes=10) ]
    TSLA_sma_pd = TSLA_sma_pd[ pd.to_datetime( TSLA_sma_pd['time'] , format = '%Y-%m-%dT%H:%M:%S.%f') >= datetime.now() - timedelta(minutes=10) ]
    AMZN_sma_pd = AMZN_sma_pd[ pd.to_datetime( AMZN_sma_pd['time'] , format = '%Y-%m-%dT%H:%M:%S.%f') >= datetime.now() - timedelta(minutes=10) ]
    FB_sma_pd = FB_sma_pd[ pd.to_datetime( FB_sma_pd['time'] , format = '%Y-%m-%dT%H:%M:%S.%f') >= datetime.now() - timedelta(minutes=10) ]
    
    
    #print(type(min_5_window + timedelta(minutes=5) ))
    #print(min_5_window + timedelta(minutes=5) )
        
    if datetime.now() >= ( min_5_window + timedelta(minutes=5) ):
        apple = AAPL_sma_pd["cost"].mean()
        google = GOOG_sma_pd["cost"].mean()
        tesla = TSLA_sma_pd["cost"].mean()
        amazon = AMZN_sma_pd["cost"].mean()
        facebook = FB_sma_pd["cost"].mean()
        
        if math.isnan(apple):
            apple = 0
        if math.isnan(google):
            google = 0
        if math.isnan(tesla):
            tesla = 0
        if math.isnan(amazon):
            amazon = 0
        if math.isnan(facebook):
            facebook = 0
        
        print_sma(apple, google, tesla, amazon, facebook)
        sma_price(apple, google, tesla, amazon, facebook)
        min_5_window = datetime.now()
        




At  2023-03-12 19:36:40.773621

-----------------------------------------------------
Simple Moving Average of instruments are as follows: 
---------------------------------------------------


,AAPL,GOOG,TSLA,AMZN,FB
0,140.21,1378.0,894.82,3269.12,231.58


-----------------------------------------------------

Message sent to topic -> profit-calculate :
---------------------------------------------
{'AAPL': 140.21, 'GOOG': 1378.0, 'TSLA': 894.82, 'AMZN': 3269.12, 'FB': 231.58, 'time': '2023-03-12T19:36:40.805890'}
---------------------------------------------




At  2023-03-12 19:41:43.367321

-----------------------------------------------------
Simple Moving Average of instruments are as follows: 
---------------------------------------------------


,AAPL,GOOG,TSLA,AMZN,FB
0,142.19,1403.76,895.63,3311.86,232.44


-----------------------------------------------------

Message sent to topic -> profit-calculate :
---------------------------------------------
{'AAPL': 142.19, 'GOOG': 1403.76, 'TSLA': 895.63, 'AMZN': 3311.86, 'FB': 232.44, 'time': '2023-03-12T19:41:43.412302'}
---------------------------------------------




At  2023-03-12 19:46:44.231897

-----------------------------------------------------
Simple Moving Average of instruments are as follows: 
---------------------------------------------------


,AAPL,GOOG,TSLA,AMZN,FB
0,145.36,1421.15,894.28,3348.73,229.02


-----------------------------------------------------

Message sent to topic -> profit-calculate :
---------------------------------------------
{'AAPL': 145.36, 'GOOG': 1421.15, 'TSLA': 894.28, 'AMZN': 3348.73, 'FB': 229.02, 'time': '2023-03-12T19:46:44.255681'}
---------------------------------------------




At  2023-03-12 19:51:44.449396

-----------------------------------------------------
Simple Moving Average of instruments are as follows: 
---------------------------------------------------


,AAPL,GOOG,TSLA,AMZN,FB
0,147.57,1416.11,890.16,3298.0,225.56


-----------------------------------------------------

Message sent to topic -> profit-calculate :
---------------------------------------------
{'AAPL': 147.57, 'GOOG': 1416.11, 'TSLA': 890.16, 'AMZN': 3298.0, 'FB': 225.56, 'time': '2023-03-12T19:51:44.473010'}
---------------------------------------------

